# Segmentation

There are different neural architectures for segmentation, but they all have the same structure:

* **Encoder** 입력 이미지에서 특징을 추출
* **Decoder** 특징을 클래스 수에 해당하는 동일한 크기와 채널 수를 가진 **마스크 이미지**로 변환

## Prerequsites

To begin with, we will import required libraries, and check if there is GPU available for training.

In [ ]:
%pip install scikit-image
%pip install torch==2.0.1 torchvision==0.15.2
%pip install tqdm

import torch
import torchvision
import matplotlib.pyplot as plt
from torchvision import transforms
from torch import nn
from torch import optim
from tqdm import tqdm # 진행 상황 표시기를 추가
import numpy as np
import torch.nn.functional as F # PyTorch의 함수형 API를 사용
from skimage.io import imread
from skimage.transform import resize
import os

# 시드 설정의 중요성
# 재현 가능성: 동일한 시드 값이 사용되면, 코드 실행 시 동일한 난수 생성 패턴이 유지됨
#    모델 훈련 결과가 일관되게 유지되도록 함
# 디버깅 용이성: 결과가 일관되면 디버깅이 훨씬 쉬워짐
#    예를 들어, 모델이 특정 에포크에서 이상한 동작을 보일 경우, 같은 시드로 다시 실행하여 그 에포크의 상태를 쉽게 조사할 수 있음
# 이 설정은 모델을 반복적으로 실행하면서 동일한 결과를 얻고자 할 때 유용
#    PyTorch와 NumPy 모두 시드 설정을 통해 난수 생성의 일관성을 유지할 수 있음

torch.manual_seed(42) # PyTorch의 난수 생성기의 시드를 42로 설정하여 실험의 재현 가능성을 보장
np.random.seed(42) # NumPy의 난수 생성기의 시드를 42로 설정

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
train_size = 0.9 # train_size: 전체 데이터셋 중 훈련에 사용할 데이터의 비율 - 0.9로 설정되었으므로 90%의 데이터를 훈련에 사용하고, 나머지 10%를 검증 또는 테스트에 사용
lr = 1e-3 # 학습률
weight_decay = 1e-6 # 가중치 감쇠 - L2 정규화 항을 통해 가중치를 줄이는 데 사용
batch_size = 32 # 훈련 중 한 번에 처리할 데이터 샘플의 수를 의미
epochs = 30 # 전체 데이터셋을 몇 번 반복하여 훈련할지를 의미

## The Dataset

- 인간 모반의 더모스코픽 이미지에 대한 PH<sup>2</sup> 데이터베이스를 사용
- 이 데이터 세트에는 전형적인 모반, 비정형 모반, 흑색종 등 세 가지 유형의 모반에 대한 200개의 이미지가 포함되어 있음
- 모든 이미지에는 모반의 윤곽을 나타내는 해당 **마스크**도 포함되어 있음

In [ ]:
# terminal에서 sudo apt-get install unrar

!wget https://www.dropbox.com/s/k88qukc20ljnbuo/PH2Dataset.rar
!unrar x -Y PH2Dataset.rar

- 이제 데이터 세트를 로드하는 코드를 정의
- 모든 이미지를 256x256 크기로 변환하고 데이터 세트를 훈련과 테스트 부분으로 분할
- 이 함수는 모반의 윤곽을 나타내는 원본 이미지와 마스크가 각각 포함된 훈련 및 테스트 데이터 세트를 반환

In [ ]:
def load_dataset(train_part, root='PH2Dataset'): # train_part 비율과 데이터가 위치한 루트 폴더를 인자로 받음
    images = []
    masks = []

    # PH2 Dataset images 폴더를 순회하면서 _Dermoscopic_Image로 끝나는 폴더에서 이미지를, _lesion으로 끝나는 폴더에서 마스크 이미지를 로드

    for root, dirs, files in os.walk(os.path.join(root, 'PH2 Dataset images')):
        if root.endswith('_Dermoscopic_Image'):
            images.append(imread(os.path.join(root, files[0])))
        if root.endswith('_lesion'):
            masks.append(imread(os.path.join(root, files[0])))

    # 로드한 이미지와 마스크를 (256, 256) 크기로 리사이즈
    # 이미지는 (N, C, H, W) 형식으로 변환하고, 마스크는 이진화하여 (N, 1, H, W) 형식으로 변환

    size = (256, 256)
    images = torch.permute(torch.FloatTensor(np.array([resize(image, size, mode='constant', anti_aliasing=True,) for image in images])), (0, 3, 1, 2))
    masks = torch.FloatTensor(np.array([resize(mask, size, mode='constant', anti_aliasing=False) > 0.5 for mask in masks])).unsqueeze(1)

    # 데이터셋을 무작위로 섞고, train_part 비율에 따라 학습용 데이터와 테스트용 데이터로 나눔

    indices = np.random.permutation(range(len(images)))
    train_part = int(train_part * len(images))
    train_ind = indices[:train_part]
    test_ind = indices[train_part:]

    # 학습 데이터셋과 테스트 데이터셋을 반환

    train_dataset = (images[train_ind, :, :, :], masks[train_ind, :, :, :])
    test_dataset = (images[test_ind, :, :, :], masks[test_ind, :, :, :])

    return train_dataset, test_dataset

train_dataset, test_dataset = load_dataset(train_size)

데이터 집합의 일부 이미지를 플롯하여 어떻게 보이는지 확인

In [ ]:
# 주어진 데이터셋에서 이미지와 마스크를 시각화하는 기능을 수행

def plotn(n, data, only_mask=False): # n은 표시할 이미지와 마스크의 수, data는 이미지와 마스크 데이터셋, only_mask는 마스크만 표시할지 여부를 결정하는 플래그
    images, masks = data[0], data[1] # data에서 이미지와 마스크를 분리
    fig, ax = plt.subplots(1, n) # 이미지를 표시할 서브플롯을 생성
    fig1, ax1 = plt.subplots(1, n) # 마스크를 표시할 서브플롯을 생성
    for i, (img, mask) in enumerate(zip(images, masks)): # 이미지를 순회하면서 표시
        if i == n: # n개까지만 표시
            break
        if not only_mask: # only_mask가 False이면 이미지를 표시
            ax[i].imshow(torch.permute(img, (1, 2, 0)))
        else: # only_mask가 True이면 첫 번째 채널만 표시
            ax[i].imshow(img[0])
        ax1[i].imshow(mask[0]) # 마스크를 표시
        ax[i].axis('off') # 축을 숨김
        ax1[i].axis('off')
    plt.show() # 플롯을 화면에 표시

plotn(5, train_dataset)

신경망에 데이터를 공급하기 위해 데이터 로더가 필요

In [ ]:
# PyTorch의 DataLoader를 사용하여 학습 데이터셋과 테스트 데이터셋을 배치로 로드

# torch.utils.data.DataLoader를 사용하여 학습 데이터셋의 데이터 로더를 만듦
# list(zip(train_dataset[0], train_dataset[1])): 학습 이미지와 마스크를 쌍으로 묶어 리스트로 만듦
# batch_size=batch_size: 배치 크기를 batch_size로 설정
# shuffle=True: 데이터셋을 섞어서 배치를 만듭니다. 이는 모델 학습 시 데이터의 순서로 인한 편향을 방지

train_dataloader = torch.utils.data.DataLoader(list(zip(train_dataset[0], train_dataset[1])), batch_size=batch_size, shuffle=True)

# torch.utils.data.DataLoader를 사용하여 테스트 데이터셋의 데이터 로더를 만듦
# list(zip(test_dataset[0], test_dataset[1])): 테스트 이미지와 마스크를 쌍으로 묶어 리스트로 만듦
# batch_size=1: 배치 크기를 1로 설정 - 이는 보통 테스트 시 각 샘플을 개별적으로 평가하기 위해 사용
# shuffle=False: 데이터셋을 섞지 않고 순서대로 배치를 만듦 - 이는 테스트 데이터의 순서를 유지하기 위해 사용

test_dataloader = torch.utils.data.DataLoader(list(zip(test_dataset[0], test_dataset[1])), batch_size=1, shuffle=False)

# train_dataloader와 test_dataloader를 튜플로 묶어 dataloaders로 저장 - 이를 통해 학습과 테스트 데이터 로더를 한 번에 관리

dataloaders = (train_dataloader, test_dataloader)

## SegNet

- 가장 간단한 인코더-디코더 아키텍처는 **SegNet**이라고 함
- 인코더에는 컨볼루션과 풀링이 포함된 표준 CNN을 사용하고, 디코더에는 컨볼루션과 업샘플링이 포함된 디컨볼루션 CNN을 사용
- 또한 다층 네트워크를 성공적으로 훈련하기 위해 배치 정규화에 의존

In [ ]:
# PyTorch를 사용하여 정의된 SegNet 아키텍처
# SegNet은 이미지 분할을 위한 신경망
# SegNet의 인코더-디코더 구조를 정의

class SegNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        # 인코더 (Encoder)
        # 이미지의 중요한 특징을 추출
        # 여러 개의 Conv2d, ReLU, BatchNorm2d, 그리고 MaxPool2d 레이어로 구성
        # 각 인코딩 단계는 컨볼루션 연산 후 활성화 함수(ReLU)와 배치 정규화를 거치고, 마지막으로 맥스 풀링을 통해 차원을 축소
        
        self.enc_conv0 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(3,3), padding=1)
        self.act0 = nn.ReLU()
        self.bn0 = nn.BatchNorm2d(16)
        self.pool0 = nn.MaxPool2d(kernel_size=(2,2))

        self.enc_conv1 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3,3), padding=1)
        self.act1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=(2,2))

        self.enc_conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3), padding=1)
        self.act2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 =  nn.MaxPool2d(kernel_size=(2,2))

        self.enc_conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3), padding=1)
        self.act3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 =  nn.MaxPool2d(kernel_size=(2,2))

        # 병목 (Bottleneck)
        # 인코더와 디코더 사이에서 가장 중요한 특징을 압축하고 전달
        # 하나의 Conv2d 레이어로 구성

        self.bottleneck_conv = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3,3), padding=1)
        
        # 디코더 (Decoder)
        # 인코더에서 추출된 특징을 사용하여 원본 이미지의 해상도와 유사하게 복원
        # 여러 개의 업샘플링(UpsamplingBilinear2d)과 Conv2d, ReLU, BatchNorm2d 레이어로 구성
        # 마지막 레이어는 컨볼루션을 통해 출력 채널을 1로 줄이고, 시그모이드 활성화 함수를 통해 결과를 이진화

        self.upsample0 =  nn.UpsamplingBilinear2d(scale_factor=2)
        self.dec_conv0 = nn.Conv2d(in_channels=256, out_channels=128, kernel_size=(3,3), padding=1)
        self.dec_act0 = nn.ReLU()
        self.dec_bn0 = nn.BatchNorm2d(128)

        self.upsample1 =  nn.UpsamplingBilinear2d(scale_factor=2)
        self.dec_conv1 =  nn.Conv2d(in_channels=128, out_channels=64, kernel_size=(3,3), padding=1)
        self.dec_act1 = nn.ReLU()
        self.dec_bn1 = nn.BatchNorm2d(64)

        self.upsample2 = nn.UpsamplingBilinear2d(scale_factor=2)
        
        self.dec_conv2 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=(3,3), padding=1)
        self.dec_act2 = nn.ReLU()
        self.dec_bn2 = nn.BatchNorm2d(32)

        self.upsample3 = nn.UpsamplingBilinear2d(scale_factor=2)
        self.dec_conv3 = nn.Conv2d(in_channels=32, out_channels=1, kernel_size=(1,1))

        self.sigmoid = nn.Sigmoid()

    # Forward 함수
    # 입력 이미지를 받아 인코더를 거친 후 병목을 통해 압축하고, 디코더를 거쳐 출력 이미지를 생성 - 이 함수는 모델의 순전파 과정을 정의

    def forward(self, x):
        e0 = self.pool0(self.bn0(self.act0(self.enc_conv0(x))))
        e1 = self.pool1(self.bn1(self.act1(self.enc_conv1(e0))))
        e2 = self.pool2(self.bn2(self.act2(self.enc_conv2(e1))))
        e3 = self.pool3(self.bn3(self.act3(self.enc_conv3(e2))))

        b = self.bottleneck_conv(e3)

        d0 = self.dec_bn0(self.dec_act0(self.dec_conv0(self.upsample0(b))))
        d1 = self.dec_bn1(self.dec_act1(self.dec_conv1(self.upsample1(d0))))
        d2 = self.dec_bn2(self.dec_act2(self.dec_conv2(self.upsample2(d1))))
        d3 = self.sigmoid(self.dec_conv3(self.upsample3(d2)))
        return d3

- Segmentation에 사용되는 손실 함수
- autoencoder에서는 두 이미지 간의 유사성을 측정해야 하며, 이를 위해 평균 제곱 오차를 사용할 수 있음
- Segmentation에서 대상 마스크 이미지의 각 픽셀은 클래스 번호(3차원을 따라 한 번 핫 인코딩됨)를 나타내므로 분류에 특화된 손실 함수인 교차 엔트로피 손실(모든 픽셀에 평균을 낸 값)을 사용
- 마스크가 2진수인 경우에는 **2진수 교차 엔트로피 손실**(BCE)을 사용

In [ ]:
# SegNet 모델을 생성하고, 최적화 함수와 손실 함수를 설정하는 과정

model = SegNet().to(device) # # SegNet 모델 생성 및 장치로 이동
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay) # # Adam 최적화 함수 설정
loss_fn = nn.BCEWithLogitsLoss() # # 이진 교차 엔트로피 손실 함수 설정 (로그잇 함수와 함께 사용)

학습 루프는 일반적인 방식으로 정의됨

In [ ]:
# SegNet 모델을 학습하는 train 함수를 정의
# 이 함수는 주어진 데이터 로더, 모델, 손실 함수, 최적화 함수 및 에포크 수를 사용하여 모델을 학습하고 검증
# 진행 상황은 tqdm 라이브러리를 통해 시각적으로 표시

def train(dataloaders, model, loss_fn, optimizer, epochs, device): # 주어진 인자를 사용하여 모델을 학습
    tqdm_iter = tqdm(range(epochs)) # 학습 진행을 시각적으로 표시하기 위해 tqdm 라이브러리를 사용
    train_dataloader, test_dataloader = dataloaders[0], dataloaders[1]

    for epoch in tqdm_iter: # 지정된 에포크 수만큼 반복
        model.train() # 모델을 학습 모드로 설정
        train_loss = 0.0 # 학습 손실 초기화
        test_loss = 0.0 # 테스트 손실 초기화

        for batch in train_dataloader: # 학습 데이터 로더에서 배치를 반복
            imgs, labels = batch
            imgs = imgs.to(device)
            labels = labels.to(device)

            preds = model(imgs) # 모델에 이미지 전달하여 예측 생성
            loss = loss_fn(preds, labels) # 손실 계산

            optimizer.zero_grad() # 옵티마이저 초기화
            loss.backward() # 역전파
            optimizer.step() # 가중치 업데이트

            train_loss += loss.item() # 배치 손실을 학습 손실에 추가

        model.eval() # 모델을 평가 모드로 설정
        with torch.no_grad(): # 검증 단계에서는 그래디언트를 계산하지 않음
            for batch in test_dataloader: # 테스트 데이터 로더에서 배치를 반복
                imgs, labels = batch
                imgs = imgs.to(device)
                labels = labels.to(device)

                preds = model(imgs)
                loss = loss_fn(preds, labels)

                test_loss += loss.item()

        train_loss /= len(train_dataloader) # 학습 손실 평균
        test_loss /= len(test_dataloader) # 테스트 손실 평균

        tqdm_dct = {'train loss:': train_loss, 'test loss:': test_loss} # 손실 값을 딕셔너리에 저장
        tqdm_iter.set_postfix(tqdm_dct, refresh=True) # tqdm 진행 표시줄을 업데이트
        tqdm_iter.refresh() # tqdm 진행 표시줄을 새로 고침

In [ ]:
# train 함수를 실행하여 SegNet 모델을 학습
# 주어진 데이터 로더, 모델, 손실 함수, 최적화 함수, 에포크 수, 그리고 장치 정보를 사용하여 학습 과정을 진행

train(dataloaders, model, loss_fn, optimizer, epochs, device)

모델을 평가하기 위해 여러 이미지에 대한 대상 마스크와 예상 마스크를 플로팅

In [ ]:
# SegNet 모델을 평가 모드로 설정하고 테스트 데이터셋에 대해 예측을 생성한 후, 일부 예측된 마스크를 시각화
# plotn 함수는 only_mask=True 설정으로 예측된 마스크만 시각화

model.eval() # 모델을 평가 모드로 설정 - 평가 모드에서는 드롭아웃 및 배치 정규화 레이어가 학습 시와 다르게 작동
predictions = [] # 리스트를 초기화
image_mask = []
plots = 5
images, masks = test_dataset[0], test_dataset[1] # images와 masks는 테스트 데이터셋에서 이미지를 가져옮
for i, (img, mask) in enumerate(zip(images, masks)): # 테스트 데이터셋에서 일부 이미지를 반복
    if i == plots: # 시각화할 이미지 수를 plots로 제한
        break
    img = img.to(device).unsqueeze(0) # 이미지를 지정된 장치로 이동시키고 배치 차원을 추가
    predictions.append((model(img).detach().cpu()[0] > 0.5).float()) # 모델에 이미지를 전달하여 예측을 생성하고, 시그모이드 활성화 함수를 적용한 후 이진화하여 리스트에 추가
    image_mask.append(mask) # 실제 마스크를 리스트에 추가
plotn(plots, (predictions, image_mask), only_mask=True) # plotn 함수를 호출하여 예측된 마스크와 실제 마스크를 시각화 - only_mask=True로 설정하여 마스크만 표시